In [1]:
from cerebro.backend import SparkBackend
from cerebro.keras import SparkEstimator

# datas storage for intermediate data and model artifacts.
from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pyspark.sql import SparkSession
import numpy as np


spark = SparkSession \
    .builder \
    .appName("Cerebro Example") \
    .getOrCreate()

...
work_dir = '/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/'
backend = SparkBackend(spark_context=spark.sparkContext, num_workers=1)
store = LocalStore(prefix_path=work_dir + 'test/')

df = spark.read.format("libsvm") \
    .option("numFeatures", "784") \
    .load("/Users/zijian/Desktop/ucsd/cse234/project/mnist/mnist.scale")

21/12/02 01:47:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


CEREBRO => Time: 2021-12-02 01:47:14, Running 1 Workers


In [2]:
df.count()

60000

In [3]:
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(dropLast=False)
encoder.setInputCols(["label"])
encoder.setOutputCols(["label_OHE"])

encoder_model = encoder.fit(df)
encoded = encoder_model.transform(df)

feature_columns=['features']
label_columns=['label_OHE']
train_df, test_df = encoded.randomSplit([0.8, 0.2], seed=100)

In [4]:
from keras_tuner.engine import hyperparameters
import autokeras as ak
from cerebro.nas.hphpmodel import HyperHyperModel

img_shape = (28, 28, 1)

input_node = ak.ImageInput()
output_node = ak.ConvBlock(
    kernel_size=hyperparameters.Fixed('kernel_size', value=3),
    num_blocks=hyperparameters.Fixed('num_blocks', value=1),
    num_layers=hyperparameters.Fixed('num_layers', value=2),
)(input_node)
output_node = ak.ClassificationHead()(output_node)
am = HyperHyperModel(input_node, output_node, seed=2000)

am.resource_bind(
    backend=backend, 
    store=store,
    feature_columns=feature_columns,
    label_columns=label_columns,
    evaluation_metric='accuracy', 
)

am.tuner_bind(
    tuner="greedy", 
    hyperparameters=None, 
    objective="val_accuracy",
    max_trials=12,
    overwrite=True,
    exploration=0.3,
)

In [5]:
rel = am.fit(train_df, epochs=4, input_shape=img_shape)

Trial 3 Complete [00h 09m 13s]
val_accuracy: 0.9746677875518799

Best val_accuracy So Far: 0.9746677875518799
Total elapsed time: 00h 26m 01s

Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_block_1/ke...|3                 |3                 
conv_block_1/se...|False             |True              
conv_block_1/ma...|True              |True              
conv_block_1/dr...|0.5               |0                 
conv_block_1/nu...|1                 |1                 
conv_block_1/nu...|2                 |2                 
conv_block_1/fi...|64                |64                
conv_block_1/fi...|128               |512               
classification_...|flatten           |flatten           
classification_...|0                 |0.25              
optimizer         |adam_weight_decay |adam              
learning_rate     |1e-05             |0.001             



Train on 1201 steps
1201/1201 [==============================] - 89s 74ms/step - batch: 600.0000 - size: 1.0000 - loss: 2.3034 - accuracy: 0.1017 1:21 - batch: 62.5000 - size: 1.0000 - loss: 2.3041 - accuracy - ETA: 1:04 - batch: 166.0000 - size: 1.0000 - loss: 2.3033 - accuracy: 0.1 - ETA: 1:04 - batch: 170.5000 - size: 1.0000 - loss: 2.3036 - accur - ETA: 1:00 - batch: 191.5000 - size: 1.0000 - loss: 2.3032 -  - ETA: 57s - batch: 209.0000 - size: 1.0000 - l - ETA: 54s - batch: 231.0000 - size: 1.0000 - loss: 2 - ETA: 52s - batch: 249.5000 - size: 1.0000 - loss: 2.3025 - accuracy:  - ETA: 51s  - ETA: 44s  - ETA: 35s - batch: 364.0000 - size: 1.0000 - ETA: 31s - batch: 393.5000 - size: 1.0000 - loss: 2 - ETA: 27s - batch: 416.0000 - size: 1.0000 - loss: 2.3029 - accuracy:  - ETA: 26s - batch: 421.5000 - size: 1.0000 - loss: 2.303 - ETA: 23s - batch: 440.5000 - size: 1.0000 - - ETA: 19s - batch: 467.5000 - size: 1.0000 - loss: 2.3030 - accuracy: 0.10 - ETA: 19s - batch: 469.0000 - size:

KeyboardInterrupt: 

1167/1201 [============================>.] - ETA: 2s - batch: 583.0000 - size: 1.0000 - loss: 2.3030 - accuracy: 0.1027

In [6]:
rel.metrics

NameError: name 'rel' is not defined